# ベクトル空間モデル・BM25

このノートブックでは，ベクトル空間モデルに基づく情報検索モデルと確率的情報検索モデルの手法であるBM25について説明します．具体的には，文書から特徴ベクトルへの変換方法，TF-IDFの計算方法，コサイン類似度による文書ランキング， BM25の計算方法，ベクトル空間モデルとBM25の比較について説明します

このノートブックでは gensim というモジュールを使います． gensimがインストールされていない場合（２個下のセルにある `import gensim` に失敗する場合）は，下記セルのコメントを外してgensimをインストールしてください．（インストールには数分かかると思います）

In [ ]:
# !conda install gensim -y

In [ ]:
import math
import numpy as np
import gensim

In [ ]:
%precision 2

# 1.コレクションの読込

4 件の文書が sample.txt に格納されています

In [ ]:
!cat sample.txt

In [ ]:
with open("sample.txt", "r") as f:  #sample.txtの読み込み
    text = f.read().strip().split("\n") #sample.txtのテキストデータを取得し，それを改行で分割
text

In [ ]:
collections = [d.split() for d in text]
collections

4件の文書があることが分かります．今回は単純のため，あらかじめ文書がトークン化されているものを扱っています．

In [ ]:
print("d1=", collections[0])
print("d2=", collections[1])
print("d3=", collections[2])
print("d4=", collections[3])

## 2. 特徴ベクトルの生成
次に文書の特徴ベクトルを生成します．ここからの流れは，以下の通りになります．

1. 文書集合（collections）から 単語->単語ID の辞書 (dictionary) を作成する．
2. 作成された辞書を基に，文書を (単語ID，出現回数）の集合 (id_collections) として表現する．
3. id_collectionsからTfidfModelを用いて，TF-IDFで重み付けされた特徴ベクトルを作成する．

まずは，文書集合（コーパス）から単語->単語ID の辞書 (dictionary) を作成します．


In [ ]:
dictionary = gensim.corpora.Dictionary(collections) # コレクションを与えて，単語->IDの辞書を作成する
dictionary.token2id #作成された辞書の中身

In [ ]:
id_collections = [dictionary.doc2bow(document) for document in collections]
id_collections

作成された id_collections は，たとえば $d_2$ をみてみると

In [ ]:
id_collections[1] #d2

という内容になっています．たとえば，(4,2)というデータは
```
単語ID4の単語（つまり，観光）が2回出現
```
という内容を表しています． 単語の出現頻度(term frequency)のみで文書を特徴ベクトル化したことになります．なお，これをnumpyのベクトルとして抽出したければ，corpus2denseメソッドを用います．

In [ ]:
tf_vectors = gensim.matutils.corpus2dense(id_collections, len(dictionary)).T
print(tf_vectors.shape) # 文書数 x 語彙数 の行列
print("d1=", tf_vectors[0])
print("d2=", tf_vectors[1])
print("d3=", tf_vectors[2])
print("d4=", tf_vectors[3])

今回用意したコーパスは語彙数が8しかありませんが，実際のケースでは，この特徴ベクトルは非常に疎になることが想像つくと思います．

さて，id_collectionsからTFIDFで重み付けされた特徴ベクトルを得るには， models.TfidfModel メソッドを用います．

In [ ]:
tfidf_model = gensim.models.TfidfModel(id_collections, normalize=False) #normalize=Trueにすると，文書長によってtfを正規化する
tfidf_corpus = tfidf_model[id_collections] #id_collectionsをtfidfで重み付けされたものに変換

In [ ]:
# 各文書のtfidfベクトルを取得
tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, len(dictionary)).T
print ("d1=", tfidf_vectors[0])
print ("d2=", tfidf_vectors[1])
print ("d3=", tfidf_vectors[2])
print ("d4=", tfidf_vectors[3])

In [ ]:
for i in range(4):
    print ("|d{}| (L2ノルム)={:.2f}".format(i+1, np.linalg.norm(tfidf_vectors[i])))

これでTF-IDFで重み付けされた特徴ベクトルが得られました．たとえば，1件目の文書 $d_1$ に対する特徴ベクトル${\mathbf d}_1$の中身を見てみます．

In [ ]:
tfidf_vectors[0]

単語IDだとよく分からないので， 単語に直して表示してみると

In [ ]:
for x in tfidf_corpus[0]:
    print("{}:{:.2f}".format(dictionary[x[0]], x[1]))  #dictionary[token_id]でアクセスすると実際の単語が返ってくる

たとえば， 文書$d_{1}$における`京都`のTFIDF値が本当に正しいのか検証してみましょう．

$tfidf_{d_2, 京都} = tf_{d_2, 京都} \ \log \frac{N}{df_{京都}}$ ,

いま， $tf_{d_2, 京都} \ = 1$, $N = 4$, $df_{京都}$ = 3 ですので，

$tfidf_{d_2, 京都} = 1 \times \log \frac{4}{3} = 0.42$
となり，gensimで得られた結果と一致していることが分かります．

In [ ]:
math.log2(4/3) # log2(4/3) の値

# 3. コサイン類似度
それでは，コサイン類似度による文書ランキングを行ってみましょう．

クエリと文書の類似度を測る前に，まずは文書同士のコサイン類似度を計算してみます． コサイン類似度の計算はgensimでも良いのですが，ここでは，実際に類似度計算に用いられるベクトルの中身が分かりやすいよう，いったんnumpyのベクトルを取得して，そのベクトルに対してコサイン類似度を計算してみます．  （参考:　numpyの(dense)行列を使うよりも， 疎行列のまま類似度を計算した方が高速）

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sim = cosine_similarity(X=[tfidf_vectors[0]], Y=[tfidf_vectors[1]])[0][0] # d1 と d2 のコサイン類似度
print("{:.2f}".format(sim))

In [ ]:
cosine_similarity(tfidf_vectors) # d1-d4の各ペアのコサイン類似度

それでは，クエリを特徴ベクトルに変換し，クエリと文書のコサイン類似度を求めていきましょう．

クエリも文書と同様にTFIDFで重み付けされた特徴ベクトルを求めます

In [ ]:
query = "観光 グルメ"
id_query = dictionary.doc2bow(query.split())
print(id_query)
query_vector = tfidf_model[id_query]
query_vector

In [ ]:
query_vector = gensim.matutils.corpus2dense([query_vector], len(dictionary)).T[0,:]
print("query_vector=", query_vector) # クエリの特徴ベクトル
query_vector

In [ ]:
tfidf_vectors

クエリと各文書とのコサイン類似度を計算してみます．

In [ ]:
cosine_similarity(X=[tfidf_vectors[1]], Y=[query_vector])[0][0] # d2との類似度

In [ ]:
cosine_similarity(X=tfidf_vectors, Y=[query_vector])[:,0] # 全ての文書とクエリのコサイン類似度

$d2, d4, d1, d3$ の順でランキングすればよい，ということが分かります．

$sim(d2, q)=0.93$ となっており， `'京都', '観光', '観光', 'グルメ'` という単語からなる $d2$ と高い類似度になるのは直感にもあいますね．

## BM25とベクトル空間モデル

BM25について説明する前にもう1つ別のクエリでもベクトル空間モデルを試して見ます

In [ ]:
query = "京都 観光"
id_query = dictionary.doc2bow(query.split())
print(id_query)
query_vector = tfidf_model[id_query]
query_vector = gensim.matutils.corpus2dense([query_vector], len(dictionary)).T
print(np.linalg.norm(query_vector))
query_vector

In [ ]:
cosine_similarity(X=tfidf_vectors, Y=query_vector)[:,0]

$d2$ が最も高い類似度なのはそのままですが， クエリ中の単語を`京都`しか含まない文書 $d3$の類似度が， `京都`と`観光`両方の単語を含む文書$d1$よりも高い値となっています． あまり好ましい結果とは言えなさそうです．

## BM25

In [ ]:
N = len(tf_vectors) # コレクション内の総文書数
N

In [ ]:
display(tf_vectors.sum(axis=1)) #　各文書の文書長（＝単語数）
avdl = tf_vectors.sum(axis=1).mean() # コレクション中の平均文書長　（＝1文書あたりの単語数の平均）
print("avdl =", avdl)

BM25を求める関数を定義しておきます
- 講義資料およびこのノートブックでは，IDFの定義として$\log\frac{N}{df}$を用いていますが， $\log\frac{N-df+0.5}{df+0.5}$がよく用いられます

In [ ]:
def bm25(N, dictionary, query, tf_vector, avdl, k1 = 1.2, b = 0.75):
    """
    BM25スコアを計算する
    
    N; 総文書数
    dictionary: gensimで得られる， 単語のDFを求めるための辞書
    query: クエリ　（ただし，　(単語ID, 出現回数) の配列）
    tf_vector: 文書のTFベクトル
    avdl: 平均文書長
    k1: パラメータ（初期値1.2）
    b: パラメータ（初期値0.75）
    """
    dl = tf_vector.sum()
    score = 0.0
    for q in query: # クエリ中に含む単語だけ考えてスコアを計算すれば良い
        i = q[0] #単語のID
        freq = q[1] # 単語の（クエリ中の）出現回数
        if tf_vector[i] == 0: # 文書に今見ている単語が出現しなかったら次の単語を見る
            continue
        df_i = dictionary.dfs[i]
        tf_i = tf_vector[i]
        idf_i = math.log2(N /df_i)
        right =  (tf_i * (k1 + 1)) / (tf_i + k1 * (1 - b + b * dl / avdl))
        s = idf_i * right * freq
        score += s
    return score

In [ ]:
query = "京都 観光"
query_vector = dictionary.doc2bow(query.split())
query_vector

In [ ]:
for i in range(len(tf_vectors)):
    score = bm25(N, dictionary, query_vector, tf_vectors[i], avdl)
    print("BM25 score: d{} = {:.2f}".format(i+1, score))

BM25でランキングすると， $d2, d1, d3, d4$ の順番になりました． ベクトル空間モデルでの結果と比べると， $d1$と$d3$の順番が入れ替わっていることに注意してください

## 補足: BM25における$k_1$の役割

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def score(tf, k1=1.2):
    s = (tf * (k1 + 1)) / (tf + k1)
    return s

In [ ]:
k1 = 1.2
x = np.linspace(0, 50, 50) # tfを0から50まで増やす
print(x)
plt.plot(x, score(x, k1))
plt.axhline((k1+1), color="red") # k1 + 1 のライン
plt.ylim(0, 2.5)
plt.xlabel("tf")
plt.show()